In [1]:
import os
import time

import re
from pathlib import Path

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


import cv2

import matplotlib.pyplot as plt
import seaborn as sns




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

print(f"Tensorflow Version: {tf.version.VERSION}")

GPU device not found
Tensorflow Version: 2.7.0


In [3]:
X = np.load('XSegmentedImagesNoBackground.npy')
Y = np.load('y2SegmentedImagesNoBackground.npy')

yUnique = np.unique(Y)
NUnique = len(yUnique)


# Update Height and Width
N, height, width, channels =  X.shape

for i_img in range(N):
  X[i_img,:,:,:] = tf.keras.applications.vgg16.preprocess_input(X[i_img,:,:,:])
# One Hot Encode Y Labels
y = tf.keras.utils.to_categorical(Y, num_classes=NUnique)

# Use a utility from sklearn to split and shuffle your dataset.
XTrain, XTest, YTrain,YTest = train_test_split(X, y,test_size=0.2)
XTrain, XVal, YTrain,YVal = train_test_split(XTrain,YTrain, test_size=0.2)

print('Training Images shape:', XTrain.shape)
print('Validation Images shape:', XVal.shape)
print('Test Images shape:', XTest.shape)


print(f"There are {NUnique} unique classes.")



Training Images shape: (105998, 100, 100, 3)
Validation Images shape: (26500, 100, 100, 3)
Test Images shape: (33125, 100, 100, 3)
There are 9 unique classes.


Create Model

In [7]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  base_model = tf.keras.applications.VGG16(
    weights="imagenet", 
    include_top=False,
    input_shape=(height, width,3),
    
  )
  # Freeze all the layers Previously Learned
  for layer in base_model.layers[:]:
      layer.trainable = False

  # Check the trainable status of the individual layers
  for layer in base_model.layers:

      print(layer, layer.trainable)

  # Define Trainable Layers
  flatten_layer = layers.Flatten()
  dense_layer = layers.Dense(1024, activation='relu')
  drop_layer = layers.Dropout(0.5)
  prediction_layer = layers.Dense(NUnique, activation='softmax')



  # Add Base Model to New Layers
  model = models.Sequential([
      base_model,
      flatten_layer,
      dense_layer,
      drop_layer,
      prediction_layer
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.categorical_crossentropy,
      metrics=metrics)

  return model

def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  fig = plt.gcf()
  fig.set_size_inches(18.5, 10.5, forward=True)
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
              linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

def plot_cm(y_true, y_pred):
  cm = confusion_matrix(y_true, y_pred)
  plt.figure(figsize=(10,10))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix')
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

In [8]:
# checkpoint
filepath="model-epoch-{epoch:02d}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)
callbacks_list = [early_stopping]

In [9]:
# Train the model for 1 epoch from Numpy data
BATCH_SIZE = 100
EPOCHS = 20


model = make_model()
history = model.fit(XTrain, YTrain, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS,
                    callbacks=callbacks_list,
                    validation_data=(XVal, YVal)
                    )

<keras.engine.input_layer.InputLayer object at 0x7fe845a3dd90> False
<keras.layers.convolutional.Conv2D object at 0x7fe83ce10940> False
<keras.layers.convolutional.Conv2D object at 0x7fe845a3dd60> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe845a3d910> False
<keras.layers.convolutional.Conv2D object at 0x7fe7fff9c6d0> False
<keras.layers.convolutional.Conv2D object at 0x7fe7fff92370> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe7fff9c280> False
<keras.layers.convolutional.Conv2D object at 0x7fe83ce13730> False
<keras.layers.convolutional.Conv2D object at 0x7fe83ce1ae80> False
<keras.layers.convolutional.Conv2D object at 0x7fe83ce21b50> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe83ce1ac70> False
<keras.layers.convolutional.Conv2D object at 0x7fe83ce27af0> False
<keras.layers.convolutional.Conv2D object at 0x7fe83ce23b50> False
<keras.layers.convolutional.Conv2D object at 0x7fe83d800190> False
<keras.layers.pooling.MaxPooling2D object at 0x7fe83d803430>

KeyboardInterrupt: 

In [ ]:
plot_metrics(history)

In [ ]:
test_predictions_baseline = model.predict(XTest, batch_size=BATCH_SIZE)

YTestPred = np.argmax(test_predictions_baseline,1)

In [ ]:
baseline_results = model.evaluate(XTest, YTest,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()
YTestAct = np.argmax(YTest,1)
plot_cm(YTestAct, YTestPred)